In [5]:
import pandas as pd
import altair as alt
import datetime

# from db import (
    # get_scenario_year_data,
    # get_all_scenarios,
    # generate_scenario_year_data,
    # get_filter_nodes_for_gate,
    # get_available_years
# )
# def load_scenario_data(scenario, year):
#     """Cache the scenario data loading to prevent unnecessary database queries"""
#     scenario_year_data = get_scenario_year_data(scenario, year)
#     scenario_data = generate_scenario_year_data(scenario_year_data)
#     return scenario_year_data, scenario_data
full_merged_df = pd.read_csv('test_vel.csv', parse_dates=["datetime"])  # Ensure 'datetime' is parsed correctly
# scenario_year_data, scenario_data = load_scenario_data(selected_model, selected_year)
# GCL = velocity
# DGL = gate closed/open
# FP2VMa = scenario for height of the water level
# Modeled_Historic = historic modeling for water level 
# in_op = gate in operation; 1 = in operation


In [ ]:
full_merged_df['week'] = full_merged_df['datetime'].dt.isocalendar().week
weekly_velocity = full_merged_df.groupby(["week", "Velocity_Category"])["time_unit"].sum().reset_index()

daily_velocity = full_merged_df.groupby(["date", "Velocity_Category"])["time_unit"].sum().reset_index()
avg_daily_velocity = pd.DataFrame(daily_velocity.groupby("Velocity_Category")['time_unit'].sum()/daily_velocity["date"].nunique()).reset_index()

daily_gate = full_merged_df.groupby(["date","DGL"])["time_unit"].sum().reset_index()
avg_daily_gate = pd.DataFrame(daily_gate.groupby("DGL")['time_unit'].sum()/daily_gate["date"].nunique()).reset_index()

daily_velocity_stats = full_merged_df.groupby(["date", "Velocity_Category"]).agg(
        unique_consecutive_groups=("consecutive_groups", "nunique"),
        total_time=("time_unit", "sum")
    ).reset_index()

daily_velocity_stats["daily_average_time_per_consecutive_group"] = (
        daily_velocity_stats["total_time"] / daily_velocity_stats["unique_consecutive_groups"]
    )
overall_daily_average_per_duration_per_velocity = daily_velocity_stats.groupby(["Velocity_Category"])['daily_average_time_per_consecutive_group'].mean().reset_index()

daily_gate_stats = full_merged_df.groupby(["date", "DGL"]).agg(
        unique_gate_count=("gate_count", "nunique"),
        total_time=("time_unit", "sum")
    ).reset_index()

daily_gate_stats["daily_average_time_per_consecutive_gate"] = (
        daily_gate_stats["total_time"] / daily_gate_stats["unique_gate_count"]
    )
overall_daily_average_per_duration_per_gate = daily_gate_stats.groupby(["DGL"])['daily_average_time_per_consecutive_gate'].mean().reset_index()
    
min_date = min(daily_velocity['date'])
max_date = max(daily_velocity['date']) 

velocity_summary_stats_title = f"Velocity summary stats from {min_date} to {max_date}."
gate_summary_stats_title = f"Gate summary stats from {min_date} to {max_date}."

velocity_summary_data = {
        "Metric": [
            f"Daily average of total amount of time velocity was {avg_daily_velocity['Velocity_Category'][0]}",
            f"Daily average of total amount of time velocity was {avg_daily_velocity['Velocity_Category'][1]}",
            f"Daily average length of consecutive hours velocity was {overall_daily_average_per_duration_per_velocity['Velocity_Category'][0]}",
            f"Daily average length of consecutive hours velocity was {overall_daily_average_per_duration_per_velocity['Velocity_Category'][1]}"
            ],
        "Hours":[
            f"{avg_daily_velocity['time_unit'][0]:.2f}",
            f"{avg_daily_velocity['time_unit'][1]:.2f}",
            f"{overall_daily_average_per_duration_per_velocity['daily_average_time_per_consecutive_group'][0]:.2f}",
            f"{overall_daily_average_per_duration_per_velocity['daily_average_time_per_consecutive_group'][1]:.2f}",
        ]}
    
gate_summary_data = {
        "Metric": [
            f"Daily average of total amount of time DGL Gate was {avg_daily_gate['DGL'][0]}",
            f"Daily average of total amount of time of DGL Gate was {avg_daily_gate['DGL'][1]}",
            f"Daily average length of consecutive hours DGL Gate was {overall_daily_average_per_duration_per_gate['DGL'][0]}",
            f"Daily average length of consecutive hours DGL Gate was {overall_daily_average_per_duration_per_gate['DGL'][1]}"
        ],
        "Hours": [
            f"{avg_daily_gate['time_unit'][0]:.2f}",
            f"{avg_daily_gate['time_unit'][1]:.2f}",
            f"{overall_daily_average_per_duration_per_gate['daily_average_time_per_consecutive_gate'][0]:.2f}",
            f"{overall_daily_average_per_duration_per_gate['daily_average_time_per_consecutive_gate'][1]:.2f}"
        ]
    }
    # Create a DataFrame
velocity_summary_df = pd.DataFrame(velocity_summary_data)
gate_summary_df = pd.DataFrame(gate_summary_data)
    # velocity_text = f"The daily average of velocity over 8ft/s over the entire time period is {avg_daily_velocity['time_unit'][0]:.2f} hours. The daily average of velocity under 8ft/s over the entire time period is {avg_daily_velocity['time_unit'][1]:.2f} hours."
    # dgl_text = f"The daily average of the DGL Gate over the entire time period is {avg_daily_gate['time_unit'][0]:.2f} hours.  The daily average of the DGL Gate closed over the entire period is {avg_daily_gate['time_unit'][1]:.2f} hours."
    

summary_stats_vel = (full_merged_df.groupby(["week", "date", "Velocity_Category"]).
        agg(
            total_velocity_duration = ("time_unit", "sum")
        )).reset_index()
summary_stats_dgl = (full_merged_df.groupby(["week", "date", "DGL"]).
        agg(
            total_gate_duration = ("time_unit", "sum")
        )).reset_index()

In [ ]:
# print(summary_stats_vel)
# print(summary_stats_dgl)

     week        date Velocity_Category  total_velocity_duration
0      17  2021-05-01        Over 8ft/s                     9.75
1      17  2021-05-01       Under 8ft/s                    14.25
2      17  2021-05-02        Over 8ft/s                    10.00
3      17  2021-05-02       Under 8ft/s                    14.00
4      18  2021-05-03        Over 8ft/s                    10.00
..    ...         ...               ...                      ...
423    47  2021-11-28       Under 8ft/s                    13.75
424    48  2021-11-29        Over 8ft/s                    10.50
425    48  2021-11-29       Under 8ft/s                    13.50
426    48  2021-11-30        Over 8ft/s                    10.00
427    48  2021-11-30       Under 8ft/s                    14.00

[428 rows x 4 columns]
     week        date     DGL  total_gate_duration
0      17  2021-05-01  Closed                16.00
1      17  2021-05-01    Open                 8.00
2      17  2021-05-02  Closed              

In [48]:
# Define a colorblind-friendly palette
# Defined using: https://colorbrewer2.org/#type=qualitative&scheme=Paired&n=4
color_palette = {
    "Velocity_Category": {"Over 8ft/s": "#a6cee3", "Under 8ft/s": "#1f78b4"},  # Blues
    "DGL": {"Closed": "#b2df8a", "Open": "#33a02c"}  # Greens
}

# Shared brush for interaction
brush = alt.selection_interval(encodings=['x'], mark=alt.BrushConfig(stroke="cyan", strokeOpacity=1))

# Velocity Bar Chart
base_vel = alt.Chart(summary_stats_vel, width=800, height=300).mark_bar().encode(
    x=alt.X("date:T", title="Date"),
    y=alt.Y("total_velocity_duration:Q", title="Hours"),
    color=alt.condition(
        brush,
        alt.Color(
            'Velocity_Category:N',
            title="Velocity Category",
            scale=alt.Scale(
                domain=list(color_palette["Velocity_Category"].keys()),
                range=list(color_palette["Velocity_Category"].values())
            )
        ),
        alt.value('lightgray')  # Gray for unselected
    ),
    tooltip=["date:T", "Velocity_Category:N", "total_velocity_duration:Q"]
).properties(
    title="Daily Velocity Over/Under 8 ft/s Duration Summary"
)

upper_vel = base_vel.mark_bar(width=alt.RelativeBandSize(0.7), stroke='grey', strokeWidth=0.5).encode(
    x=alt.X('date:T', title="Date").scale(domain=brush)
)

lower_vel = base_vel.properties(
    height=90
).add_params(brush)

vel_bar_chart = upper_vel & lower_vel

# Gate Bar Chart
base_gate = alt.Chart(summary_stats_dgl, width=800, height=300).mark_bar().encode(
    x=alt.X("date:T", title="Date"),
    y=alt.Y("total_gate_duration:Q", title="Hours"),
    color=alt.condition(
        brush,
        alt.Color(
            'DGL:N',
            title="Gate Status",
            scale=alt.Scale(
                domain=list(color_palette["DGL"].keys()),
                range=list(color_palette["DGL"].values())
            )
        ),
        alt.value('lightgray')  # Gray for unselected
    ),
    tooltip=["date:T", "DGL:N", "total_gate_duration:Q"]
).properties(
    title="Daily Gate Status Duration Summary"
)

upper_gate = base_gate.mark_bar(width=alt.RelativeBandSize(0.7), stroke='grey', strokeWidth=0.5).encode(
    x=alt.X('date:T', title="Date").scale(domain=brush)
)

lower_gate = base_gate.properties(
    height=90
).add_params(brush)

gate_bar_chart = upper_gate & lower_gate

# Combine the gate and velocity charts
combined_bar_charts = alt.vconcat(
    gate_bar_chart,
    vel_bar_chart
).resolve_scale(color='independent')

combined_bar_charts


alt.VConcatChart(...)

In [49]:
df = full_merged_df.rename(
    columns={
        "GLC": "Flow (ft/s)",
        "datetime": "Datetime",
        "Velocity_Category": "Velocity Category",
        "consecutive_groups": "Consecutive Groups",
        "min_datetime": "Flow Duration Min Datetime",
        "max_datetime": "Flow Duration Max Datetime",
        "streak_duration": "Streak Duration (hrs)",
        "gate_min_datetime": "Gate Min Datetime",
        "gate_max_datetime": "Gate Max Datetime",
        "gate_count": "Gate Count",
        "gate_streak_duration": "Gate Streak Duration (hrs)",
        "time_unit": "Time Unit (hrs)",
    }
    )
df = df.reset_index(drop=True)

In [50]:
drop_down_week = full_merged_df['week'].unique().tolist()
#print(drop_down_week)
selected_week = 17
    
    # Filter the data manually based on user input
filtered_df = full_merged_df[full_merged_df['week'] == selected_week]
    #-------------------------------------------------------------------------------------------------------
weekly_summary_stats_title = f"Daily summary stats from week {selected_week} ."
weekly_daily_velocity = filtered_df.groupby(["date", "Velocity_Category"])["time_unit"].sum().reset_index()
weekly_avg_daily_velocity = weekly_daily_velocity.groupby("Velocity_Category")['time_unit'].mean().reset_index()
    
weekly_daily_gate = filtered_df.groupby(["date","DGL"])["time_unit"].sum().reset_index()
weekly_avg_daily_gate = weekly_daily_gate.groupby("DGL")['time_unit'].mean().reset_index()
    
weekly_min_date = min(weekly_daily_velocity['date'])
weekly_max_date = max(weekly_daily_velocity['date'])

weekly_summary_data = {
        "Metric": [
            f"Daily average of total amount of time velocity was {weekly_avg_daily_velocity['Velocity_Category'][0]}",
            f"Daily average of total amount of time velocity was {weekly_avg_daily_velocity['Velocity_Category'][1]}",
            f"Daily average of total amount of time DGL Gate was {weekly_avg_daily_gate['DGL'][0]}",
            f"Daily average of total amount of time DGL Gate was {weekly_avg_daily_gate['DGL'][1]}"
        ],
        "Hours": [
            f"{weekly_avg_daily_velocity['time_unit'][0]:.2f}",
            f"{weekly_avg_daily_velocity['time_unit'][1]:.2f}",
            f"{weekly_avg_daily_gate['time_unit'][0]:.2f}",
            f"{weekly_avg_daily_gate['time_unit'][1]:.2f}"
        ]
    }

    # Create a DataFrame
weekly_summary_df = pd.DataFrame(weekly_summary_data)

print(weekly_summary_df)

                                              Metric  Hours
0  Daily average of total amount of time velocity...   9.88
1  Daily average of total amount of time velocity...  14.12
2  Daily average of total amount of time DGL Gate...  16.50
3  Daily average of total amount of time DGL Gate...   7.50


In [15]:
# import pandas as pd
full_merged_df = pd.read_csv('test_vel.csv', parse_dates=["datetime"])  # Ensure 'datetime' is parsed correctly
start_date = pd.to_datetime(full_merged_df["date"].min())
end_date = pd.to_datetime(
        full_merged_df["date"].min()
    ) + datetime.timedelta(days=7)
start_date = str(start_date)
end_date = str(end_date)
filtered_df = full_merged_df[
    (full_merged_df["date"] >= start_date)
    & (full_merged_df["date"] <= end_date)]
# filtered_df = filtered_df[filtered_df["gate_status"]==]
filtered_df.head()


,datetime,flow,stage_up,xs,velocity,location,year,Velocity_Category,consecutive_groups,min_datetime,...,streak_duration,gate_status,gate_min_datetime,gate_max_datetime,gate_count,gate_streak_duration,time_unit,week,gate,model
96,2016-05-02 00:00:00,395.204010,4.755837,53.779187,7.348642,glc,2016,Under 8ft/s,5,2016-05-02 00:00:00,...,4.75,Closed,2016-05-01 14:15:00,2016-05-02 01:30:00,46,11.5,0.25,18,glc,FPV1Ma
97,2016-05-02 00:15:00,342.860535,4.765045,53.825226,6.369886,glc,2016,Under 8ft/s,5,2016-05-02 00:00:00,...,4.75,Closed,2016-05-01 14:15:00,2016-05-02 01:30:00,46,11.5,0.25,18,glc,FPV1Ma
98,2016-05-02 00:30:00,288.430450,4.776808,53.884039,5.352799,glc,2016,Under 8ft/s,5,2016-05-02 00:00:00,...,4.75,Closed,2016-05-01 14:15:00,2016-05-02 01:30:00,46,11.5,0.25,18,glc,FPV1Ma
99,2016-05-02 00:45:00,236.642700,4.788953,53.944764,4.386759,glc,2016,Under 8ft/s,5,2016-05-02 00:00:00,...,4.75,Closed,2016-05-01 14:15:00,2016-05-02 01:30:00,46,11.5,0.25,18,glc,FPV1Ma
100,2016-05-02 01:00:00,187.287033,4.803263,54.016314,3.467231,glc,2016,Under 8ft/s,5,2016-05-02 00:00:00,...,4.75,Closed,2016-05-01 14:15:00,2016-05-02 01:30:00,46,11.5,0.25,18,glc,FPV1Ma


d

In [ ]:
hydro_df = pd.read_csv("test_hydro.csv")
# print(hydro_df.head())
start_date = pd.to_datetime(hydro_df["datetime"].min())
end_date = pd.to_datetime(
        hydro_df["datetime"].min()
    ) + datetime.timedelta(days=7)
start_date = str(start_date)
end_date = str(end_date)
filtered_hydro_df = hydro_df[
    (hydro_df["datetime"] >= start_date)
    & (hydro_df["datetime"] <= end_date)]

# print(filtered_hydro_df.tail())

          id  scenario_id             datetime node  param  water_level  unit  \
668  7523774            1  2016-05-07 23:00:00  dgl  stage     5.815108  FEET   
669  7523775            1  2016-05-07 23:15:00  dgl  stage     5.793856  FEET   
670  7523776            1  2016-05-07 23:30:00  dgl  stage     5.771817  FEET   
671  7523777            1  2016-05-07 23:45:00  dgl  stage     5.749820  FEET   
672  7523778            1  2016-05-08 00:00:00  dgl  stage     5.727697  FEET   

                     updated_at  year  time_unit  week  
668  2025-02-10 19:34:56.350842  2016       0.25    18  
669  2025-02-10 19:34:56.350842  2016       0.25    18  
670  2025-02-10 19:34:56.350842  2016       0.25    18  
671  2025-02-10 19:34:56.350842  2016       0.25    18  
672  2025-02-10 19:34:56.350842  2016       0.25    18  


In [28]:
elev_hist = alt.Chart(filtered_hydro_df).transform_joinaggregate(
    total = 'count(*)'
).transform_calculate(
    pct = '1/datum.total' 
).mark_bar(
    color="#1f78b4",
    opacity=0.7
).encode(
    x=alt.X("water_level:Q", bin=True, title="ft NAVD88"),
    y=alt.Y("sum(pct):Q", title="Percentage", axis=alt.Axis(format='%')),
    tooltip = alt.Tooltip ("sum(pct):Q", format="%")
).properties(
    title="Daily Minimum Stage",
    width=600,
    height=400
)
elev_hist


alt.Chart(...)

In [7]:
# test = filtered_df[filtered_df['streak_duration'] == 5.5]
# test
# print(unique_streaks)
# v_hist = alt.Chart(filtered_df).transform_joinaggregate(
#     total = 'count(*)'
# ).transform_calculate(
#     pct = '1/datum.total' 
# ).mark_bar(
#     color="#1f78b4",
#     opacity=0.7
# )
filtered_df = filtered_df[filtered_df["Velocity_Category"] == "Over 8ft/s"]
# print(filtered_df)
unique_streaks = filtered_df.groupby(["min_datetime", "streak_duration"]).size().reset_index(name="count").drop("count", axis=1)
print(unique_streaks)
# streak_duration_counts = unique_streaks["streak_duration"].value_counts().reset_index()
# streak_duration_counts.columns = ["streak_duration", "count"]
# filtered_df.head()
# print(streak_duration_counts)
streak_duration_hist = alt.Chart(unique_streaks).transform_joinaggregate(
    total = 'count(*)'
).transform_calculate(
    pct = '1/datum.total' 
).mark_bar(
    color="#1f78b4",
    opacity=0.7
).encode(
    x=alt.X("streak_duration:Q", bin=True, title="Streak Duration"),
    y=alt.Y("sum(pct):Q", title="Percentage", axis=alt.Axis(format='%')),
    tooltip = alt.Tooltip ("sum(pct):Q", format="%")
).properties(
    title="Consecutive number of hours above 8ft/s",
    width=600,
    height=400
)
streak_duration_hist


           min_datetime  streak_duration
0   2016-05-02 04:45:00             7.00
1   2016-05-02 16:00:00             8.50
2   2016-05-03 05:15:00             7.50
3   2016-05-03 17:15:00             7.25
4   2016-05-04 05:45:00             9.50
5   2016-05-04 18:15:00             7.00
6   2016-05-05 06:15:00             9.25
7   2016-05-05 19:00:00             7.25
8   2016-05-06 06:45:00             9.75
9   2016-05-06 20:00:00             6.75
10  2016-05-07 07:15:00            10.25
11  2016-05-07 21:00:00             6.25
12  2016-05-08 08:15:00            10.75
13  2016-05-08 22:15:00             5.50


alt.Chart(...)

In [120]:
import numpy as np
# min(filtered_df.velocity)transform_calculate(
#     FlowVelocityDuration="'Flow ' + datum.Velocity_Category + ' duration is ' + datum.streak_duration + ' hours'",
#     GateStatusDuration="'Gate ' + datum.gate_status + ' duration is ' + datum.gate_streak_duration + ' hours'"
# )
print(filtered_df.year.unique())
xrule = alt.Chart(filtered_df).mark_rule(
    color="red", 
    strokeDash=[12, 6], 
    size=1.5
).encode(
    x=alt.datum(8)
).properties(
    width=300,
    height=300
)
v_hist = alt.Chart(filtered_df).transform_joinaggregate(
    total = 'count(*)'
).transform_calculate(
    pct = '1/datum.total' 
).mark_bar(
    color="#1f78b4",
    opacity=0.7
).encode(
    alt.X('velocity:Q', title = "Velocity (ft/s)", bin=True),
    alt.Y('sum(pct):Q', title ="Percent of Total Time", axis=alt.Axis(format='%')),
    # alt.Color('location:N'),
    alt.Tooltip ("sum(pct):Q", format="%"),
    column="location:N"
)
v_hist
# (v_hist + xrule).facet("year:N")
# layered_chart = alt.layer(v_hist, xrule).properties(
#     width=700,
#     height=400,
#     title=alt.Title(text='Velocity Through Fish Passage', align='center')
# )
# layered_chart

[2016]


alt.Chart(...)

In [17]:
# Define a colorblind-friendly palette
color_palette = {
    "Velocity_Category": {"Over 8ft/s": "#a6cee3", "Under 8ft/s": "#1f78b4"},  # Blues
    "gate_status": {"Closed": "#b2df8a", "Open": "#33a02c"}  # Greens
}

gate = filtered_df["gate"].unique()[0]
model = filtered_df["model"].unique()[0]
shared_y_scale = alt.Scale(domain=[-5, 17])

# Selection interval for interactivity
interval = alt.selection_interval(
    encodings=['x'], 
    mark=alt.BrushConfig(fill='blue')
)

# Base chart for velocity
base = alt.Chart(filtered_df).mark_line(color=color_palette["Velocity_Category"]["Under 8ft/s"]).encode(
    x=alt.X(
        'yearmonthdatehoursminutes(datetime):T', 
        title='Datetime', 
        axis=alt.Axis(format='%b %d, %Y', labelAngle=-45, title='Date')
    ),
    y=alt.Y('velocity:Q', title='Velocity (ft/s)'),
).add_params(interval).properties(
    title=f"Fish Passage Velocity and {gate} Gate Status Zoomed",
    height=300
)

# Closed gates rects with updated color
closed_gates = filtered_df[['gate_min_datetime', 'gate_max_datetime', 'gate_status']].drop_duplicates().reset_index(drop=True)
color_scale = alt.Scale(
    domain=["Closed"], range=[color_palette["gate_status"]["Closed"]]
)
area_dgl_true = alt.Chart(closed_gates).mark_rect().encode(
    x='gate_min_datetime:T',
    x2='gate_max_datetime:T',
    color=alt.Color(
                "gate_status:N",
                scale=color_scale,
                legend=alt.Legend(title="Gate Status"),
            ),
    opacity=alt.value(0.2),
).transform_filter(
    alt.datum.gate_status == "Closed"
)

# Rule for the velocity threshold (8 ft/s)
yrule = alt.Chart(filtered_df).mark_rule(
    color="red", 
    strokeDash=[12, 6], 
    size=1.5
).encode(
    y=alt.datum(8)
).properties(
    width=300,
    height=300
)

# Nearest point interaction
nearest = alt.selection_point(nearest=True, on="pointerover", fields=["datetime"], empty=False)

# Points for interaction
points = base.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)

# Rules and tooltips
rules = alt.Chart(filtered_df).transform_calculate(
    FlowVelocityDuration="'Flow ' + datum.Velocity_Category + ' duration is ' + datum.streak_duration + ' hours'",
    GateStatusDuration="'Gate ' + datum.gate_status + ' duration is ' + datum.gate_streak_duration + ' hours'"
).mark_rule(color="gray").encode(
    x="datetime:T",
    opacity=alt.condition(nearest, alt.value(0.3), alt.value(0)),
    tooltip=[
        alt.Tooltip('yearmonthdatehoursminutes(datetime):T', title='Datetime'),
        alt.Tooltip('GLC:Q', title="Velocity (ft/s)", format=".2f"),
        alt.Tooltip('FlowVelocityDuration:N', title="Flow Velocity Duration"),
        alt.Tooltip('GateStatusDuration:N', title="Gate Status Duration")
    ]
).add_params(nearest)

# Velocity summary text
vel_text = alt.Chart(filtered_df).mark_text(align='right').encode(
    y=alt.Y('stat:N', axis=None),
    text=alt.Text('summary:N')
).transform_filter(interval).transform_aggregate(
    max_velocity='max(velocity)',
    min_velocity='min(velocity)',
    avg_velocity='mean(velocity)'
).transform_fold(
    ['max_velocity', 'min_velocity', 'avg_velocity'],  
    as_=['stat', 'value']
).transform_calculate(
    summary='datum.stat + ": " + format(datum.value, ".2f")'
)
velocity = vel_text.encode(text='summary:N').properties(
    title=alt.Title(text='Flow Velocity Summary', align='center')
)

# Velocity duration summary text
vel_duration_text = alt.Chart(filtered_df).mark_text(align='right').encode(
    y=alt.Y('Velocity_Category:N', axis=None),
    text=alt.Text('summary:N')
).transform_filter(interval).transform_aggregate(
    total_time='sum(time_unit)',
    groupby=["Velocity_Category"]
).transform_fold(
            ["total_time"],  # Separate each statistic
            as_=["stat", "value"],
).transform_calculate(
    summary='datum.Velocity_Category + ": " + format(datum.total_time, ".2f") + " hours"'
)
velocity_duration = vel_duration_text.encode(text="summary:N").properties(
    title=alt.Title(text="Velocity Duration Summary", align="center")
)
# velocity_duration = vel_duration_text.properties(
#     title=alt.Title(text='Velocity Duration Summary', align='center')
# )

# Gate duration summary text
gate_duration_text = alt.Chart(filtered_df).mark_text(align='right').encode(
    y=alt.Y('gate_status:N', axis=None),
    text=alt.Text('summary:N')
).transform_filter(interval).transform_aggregate(
    total_time='sum(time_unit)',
    groupby=["gate_status"]
).transform_fold(
            ["total_time"],  # Separate each statistic
            as_=["stat", "value"],
).transform_calculate(
    summary='datum.gate_status + ": " + format(datum.total_time, ".2f") + " hours"'
)
gate_duration = gate_duration_text.properties(
    title=alt.Title(text='Gate Status Duration Summary', align='center')
)

# Combine all components
layered_chart = alt.layer(base, points, yrule, rules, area_dgl_true).properties(
    width=700,
    height=400
)

velocity = velocity.properties(width=200, height=100)
velocity_duration = velocity_duration.properties(width=200, height=100)
gate_duration = gate_duration.properties(width=200, height=100)

horizontal_text_summary = alt.hconcat(velocity, velocity_duration)
text_summary = alt.vconcat(velocity, velocity_duration, gate_duration)

combined_chart = alt.hconcat(
    layered_chart,
    text_summary
)

combined_chart


alt.HConcatChart(...)

In [52]:
elev_cols = ["FP2VMa", "Modeled_Historic"]

base_elevation = alt.Chart(filtered_df).mark_line().encode(
        x=alt.X('yearmonthdatehoursminutes(datetime):T', title='Date', axis=alt.Axis(format='%b %d, %Y', labelAngle=-45)),
        y=alt.Y('value:Q', title='Feet'),
        color='model:N'
    ).transform_fold(
        ['FP2VMa', 'Modeled_Historic'],  # Columns to be "melted" into a long format
        as_=['model', 'value']  # New column names
    ).add_params(
        interval
    ).properties(
        title="Weekly Summary of Minimum Stage At DGL"
    )
yrule_wl = alt.Chart(filtered_df).mark_rule(color = "purple", strokeDash=[12, 6], size=1.5).encode(
            y=alt.datum(2.3)
    )
yrule_wl_text = alt.Chart(filtered_df).mark_text(
        text="Water Level Compliance",
        align="left",
        baseline="bottom",
        fontSize=12,
        color="grey",
        dx=5  # Offset text slightly to the right of the rule
    ).encode(
        y=alt.datum(2.3)  # Same y position as the rule
    )
nearest_elev = alt.selection_point(nearest=True, on="pointerover",
                                  fields=["datetime"], empty=False)
points_elev = base_elevation.mark_point().encode(
        opacity=alt.condition(nearest_elev, alt.value(1), alt.value(0))
    )
rules_elev = alt.Chart(filtered_df).mark_rule(color="gray", opacity=0).encode(
        x="datetime:T",
        opacity=alt.condition(nearest_elev, alt.value(0.3), alt.value(0)),
    ).add_params(nearest_elev)
when_near = alt.when(nearest_elev)
text = base_elevation.mark_text(
        align="left", dx=5, dy=-5
    ).transform_calculate(
        label='format(datum.value, ".2f") + " feet"'
    ).encode(
        text=when_near.then("label:N").otherwise(alt.value(" "))
    )
average_scenario_stage = alt.Chart(filtered_df).mark_text(align='right').encode(
            y=alt.Y('stat:N', axis=None),
            text=alt.Text('summary:N')
        ).transform_filter(
            interval
        ).transform_aggregate(
            average_stage='mean(FP2VMa)'
        ).transform_fold(
            ['average_stage'],  # Separate each statistic
            as_=['stat', 'value']
        ).transform_calculate(
            summary='format(datum.average_stage, ".2f") + " feet"'
        )
avg_stage = average_scenario_stage.encode(text='summary:N').properties(
            title=alt.Title(text='Average FP2VMa Minimum Stage', align='center')
    )
scenario_duration_below_wl = alt.Chart(filtered_df).mark_text(align='right').encode(
        y=alt.Y('wl_stat:N', axis=None),
            text=alt.Text('below_wl:N')
        # ).transform_filter(
        #     interval
        ).transform_filter(
            "datum.FP2VMa < 2.3"
        ).transform_aggregate(
            total_time_below_wl='sum(time_unit)'
        ).transform_calculate(
            below_wl='format(datum.total_time_below_wl, ".2f") + " hour"'
        ).properties(
            title=alt.Title(text='Scenario Below Water Level Compliance', align='center')
        )

modeled_historic_below_wl = alt.Chart(filtered_df).mark_text(align='right').encode(
        y=alt.Y('wl_stat:N', axis=None),
            text=alt.Text('below_wl:N')
        ).transform_filter(
            interval
        ).transform_filter(
            "datum.Modeled_Historic < 2.3"
        ).transform_aggregate(
            total_time_below_wl='sum(time_unit)'
        ).transform_calculate(
            below_wl='format(datum.total_time_below_wl, ".2f") + " hour"'
        ).properties(
            title=alt.Title(text='Modeled Historic Below Water Level Compliance', align='center')
        )
    
avg_stage = avg_stage.properties(width=200, height=100)
scenario_duration_below_wl = scenario_duration_below_wl.properties(width=200, height=100)
modeled_historic_below_wl = modeled_historic_below_wl.properties(width=200, height=100)
    # (base, points, yrule, rules, area_dgl_true
weekly_min_stage_chart = alt.layer(base_elevation, 
                              points_elev, 
                              yrule_wl, 
                              yrule_wl_text,
                              rules_elev,
                              area_dgl_true,
                              text
    ).properties(width=700, height=400)
    
combined_elev_text = alt.vconcat(
        avg_stage,
        scenario_duration_below_wl,
        modeled_historic_below_wl
    )
combined_elev_chart = alt.hconcat(
        weekly_min_stage_chart,
        combined_elev_text
    )

combined_elev_chart

alt.HConcatChart(...)